#### Prepared for Gabor's Data Analysis

### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.

### CHAPTER 21
**CH20A Founder/family ownership and quality of management**

using the wms-management dataset

version 1.0 2021-05-05

In [1]:
import os
import sys
import warnings

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from IPython.core.display import HTML
from sklearn.neighbors import NearestNeighbors
from stargazer.stargazer import Stargazer

warnings.filterwarnings("ignore")

In [2]:
# Current script folder
current_path = os.getcwd()
dirname = current_path.split("da_case_studies")[0]

# location folders
data_in = dirname + "da_data_repo/wms-management-survey/clean/"
data_out = dirname + "da_case_studies/ch21-ownership-management-quality/"
output = dirname + "da_case_studies/ch21-ownership-management-quality/output/"

func = dirname + "da_case_studies/ch00-tech-prep/"
sys.path.append(func)

In [3]:
# Import the prewritten helper functions
from py_helper_functions import *

### Read in data

In [4]:
data = pd.read_csv(data_out + "wms_da_textbook-work.csv", index_col=0)

In [5]:
data.groupby("foundfam_owned")[["management"]].mean()

,management
foundfam_owned,
0.0,3.047261
1.0,2.681602


### Set variables to use

In [6]:
y_var = "management"
x_var = "foundfam_owned"

control_vars = [
    "degree_nm",
    "degree_nm_sq",
    "compet_moder",
    "compet_strong",
    "lnemp",
    "age_young",
    "age_old",
    "age_unknown",
]

control_vars_to_interact = ["industry", "countrycode"]

## Regressions

In [7]:
# OLS with no control vars.
formula1 = y_var + " ~ " + x_var
ols1 = smf.ols(formula=formula1, data=data).fit()

In [8]:
# OLS with all control vars
formula2 = (
    y_var + " ~ " + x_var + " + " + " + ".join(control_vars + control_vars_to_interact)
)
ols2 = smf.ols(formula=formula2, data=data).fit()

In [9]:
# OLS with all controls + interactions
formula3 = (
    y_var
    + " ~ "
    + x_var
    + "+"
    + ":".join(control_vars_to_interact)
    + "+("
    + "+".join(control_vars)
    + ")*("
    + "+".join(control_vars_to_interact)
    + ")"
)
ols3 = smf.ols(formula=formula3, data=data).fit()

In [10]:
stargazer = Stargazer([ols1, ols2, ols3])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer.covariate_order([x_var, "Intercept"])
stargazer.significant_digits(2)
stargazer.custom_columns(
    ["No confounders", "With confounders", "With confounders interacted"],
    [1, 1, 1],
)
stargazer.show_model_numbers(False)
HTML(stargazer.render_html())

## Exact matching

In [11]:
data["management"].describe()

count    8439.000000
mean        2.881352
std         0.636060
min         1.055556
25%         2.444444
50%         2.888889
75%         3.333333
max         4.888889
Name: management, dtype: float64

In [12]:
data["empbin5"] = pd.cut(
    data["emp_firm"],
    bins=data["emp_firm"].quantile(np.arange(0, 1.01, 0.2)),
    labels=["1", "2", "3", "4", "5"],
    include_lowest=True,
    right=False,
).fillna("5")
data["agecat"] = (
    (data["age_young"] == True)
    + 2 * (data["age_mid"] == True)
    + 3 * (data["age_old"] == True)
    + 4 * (data["age_unknown"] == True)
)

In [13]:
data["n"] = 1
data["n1"] = data["foundfam_owned"]
data["n0"] = 1 - data["foundfam_owned"]
data["y0"] = np.where(data["foundfam_owned"] == 0, data["management"], None)
data["y1"] = np.where(data["foundfam_owned"] == 1, data["management"], None)

In [14]:
data_agg = (
    data.groupby(
        [
            "degree_nm_bins",
            "agecat",
            "competition",
            "empbin5",
            "industry",
            "countrycode",
        ]
    )
    .agg({"n": "sum", "n0": "sum", "n1": "sum", "y0": "mean", "y1": "mean"})
    .reset_index()
)

In [15]:
# theoretical combinations
data_agg.shape[0]

115200

In [16]:
# combinations in the data
data_agg = data_agg.dropna(subset=["y0", "y1"], how="all")
data_agg.shape[0]

6976

In [17]:
# firms with exact match
data_agg.loc[((data_agg["n0"] >= 1) & (data_agg["n1"] >= 1)), "n"].sum()

1207

In [18]:
# firms with no exact match
data_agg.loc[((data_agg["n0"] == 0) | (data_agg["n1"] == 0)), "n"].sum()

7232

In [19]:
# random order just for the examples
np.random.seed(123)
data_sample = data_agg.sample(n=340)[
    [
        "industry",
        "countrycode",
        "degree_nm_bins",
        "competition",
        "agecat",
        "empbin5",
        "n1",
        "n0",
        "n",
    ]
]

In [20]:
# examples with founder/family only
data_sample.loc[lambda x: (x["n1"] == 1) & (x["n0"] == 0)].head(8)

,industry,countrycode,degree_nm_bins,competition,agecat,empbin5,n1,n0,n
38827,textile,pt,"[0.001, 0.05)",10+ competitors,2,1,1.0,0.0,1
31606,rubber,tr,"[0.001, 0.05)",10+ competitors,1,1,1.0,0.0,1
113186,rubber,br,"[0.2, 1.01)",5-9 competitors,4,1,1.0,0.0,1
59219,ind_machinery,in,"[0.05, 0.2)",0-4 competitors,1,4,1.0,0.0,1
57704,food,gb,"[0.05, 0.2)",0-4 competitors,1,1,1.0,0.0,1
96704,leather,gb,"[0.2, 1.01)",10+ competitors,2,2,1.0,0.0,1
2263,primary_metal,fr,"[0.0, 0.001)",0-4 competitors,1,5,1.0,0.0,1
69482,printing,br,"[0.05, 0.2)",10+ competitors,2,5,1.0,0.0,1


In [21]:
# examples with other only:
data_sample.loc[lambda x: (x["n1"] == 0) & (x["n0"] == 1)].head(8)

,industry,countrycode,degree_nm_bins,competition,agecat,empbin5,n1,n0,n
58584,chemical,ar,"[0.05, 0.2)",0-4 competitors,1,3,0.0,1.0,1
13581,furniture,sw,"[0.0, 0.001)",5-9 competitors,2,4,0.0,1.0,1
95123,fabricated_metal,in,"[0.2, 1.01)",0-4 competitors,2,4,0.0,1.0,1
18820,food,cl,"[0.0, 0.001)",10+ competitors,3,5,0.0,1.0,1
94741,ind_machinery,it,"[0.2, 1.01)",0-4 competitors,2,3,0.0,1.0,1
7866,ind_machinery,po,"[0.0, 0.001)",0-4 competitors,2,2,0.0,1.0,1
6355,food,pt,"[0.0, 0.001)",5-9 competitors,1,4,0.0,1.0,1
41493,instrument,sw,"[0.001, 0.05)",5-9 competitors,2,2,0.0,1.0,1


In [22]:
# examples of similar firms unmatched
data_sample.loc[
    lambda x: (x["countrycode"] == "us") & (x["industry"] == "food") & (x["n"] == 1)
]

,industry,countrycode,degree_nm_bins,competition,agecat,empbin5,n1,n0,n
71159,food,us,"[0.05, 0.2)",5-9 competitors,2,4,0.0,1.0,1
51479,food,us,"[0.001, 0.05)",0-4 competitors,4,3,0.0,1.0,1


In [23]:
# ATE/ATET by exact matching:
data_agg["d"] = data_agg["y1"] - data_agg["y0"]
filtered_df = data_agg[(data_agg["n0"] > 0) & (data_agg["n1"] > 0)]

ATE = np.average(filtered_df["d"], weights=filtered_df["n"])
ATET = np.average(filtered_df["d"], weights=filtered_df["n1"])

summary = pd.DataFrame({"ATE": [ATE], "ATET": [ATET]})
print(summary)

        ATE      ATET
0 -0.155337 -0.145756


## Matching on the propensity score 

In [24]:
from psm import psmatch

data_pscore = data[[y_var] + [x_var] + control_vars + control_vars_to_interact].dropna()
formula_pscore1 = x_var + "~" + "+".join(control_vars + control_vars_to_interact)

result1 = psmatch(data_pscore, x_var, y_var, formula_pscore1, n_bootstraps=200)

In [25]:
formula_pscore2 = (
    x_var
    + " ~ "
    + ":".join(control_vars_to_interact)
    + "+("
    + "+".join(control_vars)
    + ")*("
    + "+".join(control_vars_to_interact)
    + ")"
)

result2 = psmatch(data_pscore, x_var, y_var, formula_pscore2, n_bootstraps=50)

In [26]:
def format_psmatch_results(r, title: str, round_digits=2):
    r = [str(np.round(r, round_digits)) for r in r]
    return pd.DataFrame(
        {
            "": ["ATE estimate", "", "ATET estimate", ""],
            title: [r[4], "(" + r[5] + ")", r[0], "(" + r[1] + ")"],
        }
    ).set_index("")


print(
    pd.concat(
        [
            format_psmatch_results(result1, "All confounders"),
            format_psmatch_results(
                result2, "All confounders with industry and country"
            ),
        ],
        axis=1,
    )
)
print(
    """
    Note: Bootstrapped standard errors are in parentheses."""
)
print(
    """
    Note: Estimates are different for `All confounders
    with industry and country` model than in book. This is because 
    Stata drops variables and observations because of 
    multicollinearity, while Python uses `l1` regularization.
    """
)

              All confounders All confounders with industry and country
                                                                       
ATE estimate            -0.18                                     -0.18
                       (0.02)                                    (0.02)
ATET estimate            -0.2                                     -0.19
                       (0.02)                                    (0.03)

    Note: Bootstrapped standard errors are in parentheses.

    Note: Estimates are different for `All confounders
    with industry and country` model than in book. This is because 
    Stata drops variables and observations because of 
    multicollinearity, while Python uses `l1` regularization.
    


## Check common support

In [27]:
# Cross-tabulations for competition, industry, country

combined = pd.concat(
    [
        pd.crosstab(data["foundfam_owned"], data[col], normalize="index")
        for col in ["compet_moder", "compet_strong", "industry", "countrycode"]
    ],
    axis=1,
)
combined

,False,True,False,True,apparel,chemical,electronic,fabricated_metal,food,furniture,...,jp,mx,ni,nz,po,pt,sg,sw,tr,us
foundfam_owned,,,,,,,,,,,,,,,,,,,,,
0.0,0.772885,0.227115,0.484382,0.515618,0.021041,0.111280,0.088937,0.082863,0.113883,0.018655,...,0.015618,0.036009,0.012798,0.019523,0.028200,0.018004,0.038178,0.041866,0.021258,0.110629
1.0,0.790807,0.209193,0.423870,0.576130,0.057979,0.077305,0.048838,0.085662,0.147297,0.036041,...,0.009924,0.059023,0.012536,0.013058,0.009663,0.026639,0.022983,0.009141,0.060329,0.050666


In [28]:
# Group by 'foundfam_owned' and get descriptive statistics for 'degree_nm'
degree_summary = (
    data.groupby("foundfam_owned")["degree_nm"]
    .describe(percentiles=[0.01, 0.05, 0.95, 0.99])
    .reset_index()
)
degree_summary

,foundfam_owned,count,mean,std,min,1%,5%,50%,95%,99%,max
0,0.0,4610.0,0.118180,0.172951,0.0,0.0,0.0,0.05,0.5,0.8,1.0
1,1.0,3829.0,0.088221,0.144737,0.0,0.0,0.0,0.03,0.4,0.7,1.0


In [29]:
# Group by 'foundfam_owned' and get descriptive statistics for 'emp_firm'
employment_summary = (
    data.groupby("foundfam_owned")["emp_firm"]
    .describe(percentiles=[0.01, 0.05, 0.95, 0.99])
    .reset_index()
)